# Prediction of User Reactions
We want to predict:
* How to get high score/consensus (upvotes - downvotes)
* How to be controversial (upvotes + downvotes)  
in the comments.

### Import data

In [103]:
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
# Not necessary, but I like the ggplot style better
plt.style.use('ggplot')

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df_art = pd.read_csv('articles_2017_08.csv')
df_com = pd.read_csv('comments_2017_08.csv').head(20000) # crop because battery life, skews data
# Make float better readable
pd.options.display.float_format = '{:.3f}'.format

### Prepare features

In [104]:
from datetime import datetime

def get_dt_obj(time):
    time = time.replace('am ', '')
    # Make datetime object from string
    return datetime.strptime(time, '%d.%m.%Y %H:%M')

def time_since_epoch(time):
    return (get_dt_obj(time)-datetime(1970,1,1)).total_seconds()

def get_hour_of_day(time):
    return get_dt_obj(time).hour

def get_weekday(time):
    return get_dt_obj(time).weekday()

# Basically same as "the hour of week" or "weekday_hourOfDay"
def get_weekday_float(time):
    hour = float(get_hour_of_day(time))
    weekday = get_weekday(time)
    return float(weekday) + hour / 24

def get_weekday_hour(time):
    return '{}_{}'.format(get_weekday(time), get_hour_of_day(time))

df_com['time_since_epoch'] = df_com['time'].apply(time_since_epoch)
df_com['hour'] = df_com['time'].apply(get_hour_of_day)
df_com['weekday'] = df_com['time'].apply(get_weekday) # 0 = Monday
df_com['weekday_fl'] = df_com['time'].apply(get_weekday_float)
df_com['weekday_hour'] = df_com['time'].apply(get_weekday_hour)
df_com['is_answer'] = df_com['tit'].apply(lambda x: str(x).startswith('@'))
df_com['con_len'] = df_com['con'].apply(lambda x: len(x))
df_com['con_num_words'] = df_com['con'].apply(lambda x: len(x.split()))
df_com['score'] = df_com['vup'] - df_com['vdo']
df_com['contr'] = df_com['vup'] + df_com['vdo']

df_com['tit'] = df_com['tit'].str.lower()
df_com['con'] = df_com['con'].str.lower()

def get_category(link):
    t = link.split('/')
    if len(t) <= 1:
        return ''
    else:
        return t[1]

df_art['cat'] = df_art['link'].apply(get_category)
df_art['header_len'] = df_art['header'].apply(lambda x: len(x))
df_art['text_len'] = df_art['text'].apply(lambda x: len(str(x)))
df_art['text_num_words'] = df_art['text'].apply(lambda x: len(str(x).split()))

# Left inner join
df_merge = pd.merge(left=df_com, right=df_art, left_on='tId', right_on='tId')

# is empty: (= all comments correspond to an article)
#print(df_merge[pd.isnull(df_merge['link'])])
#df_merge[df_merge['vup']>1000]

# Get order of comments per article
df_merge_art = df_merge.sort_values(['tId', 'time_since_epoch']).groupby('tId')
# Get time since the first comment
def get_time_since_first(group):
    first = group.iloc[:1]['time_since_epoch']
    group['time_since_first'] = group['time_since_epoch'].apply(lambda x: (x - first) / 3600)
    # Remove those very late comments, after x hours
    group = group[group['time_since_first'] < 36]
    return group

df_merge_art = df_merge_art.apply(get_time_since_first)


### Trying to remove skew

In [111]:
import math
def show_hist(col):
    df_merge_art['bins'] = pd.cut(df_merge_art[col], 100)
    h = df_merge_art.groupby(['bins']).size().plot.bar(title=col)
    plt.gca().get_xaxis().set_visible(False)
    plt.show()
    
# This categories are random
bins = [-np.inf, 2, 25, np.inf]
labels = ['<0', 'small', 'big']
df_merge_art['score_cat'] = pd.cut(df_merge_art['score'], bins, labels=labels)
print(df_merge_art.groupby('score_cat').size())

#df = df[(df['score']<2) | (df['score']>=25)]
    
def sgn(x):
    if x == 0: return 0
    else: return x/abs(x)
# Removes left/right skew 
transformation = lambda x: sgn(x)*math.log(abs(x) + 1)
for col in ['weekday_fl', 'con_len', 'text_len', 'time_since_first', 
            'score', 'contr', 'header_len', 'con_num_words', 'text_num_words']:
    #min_val = df_merge_art[col].min()
    df[col + '_trans'] = df_merge_art[col].apply(transformation)
    #show_hist(col + '_trans')

# I removed very weak correlations to "score" and "contr"
cols = ['score_trans', 'contr', 'weekday', 'hour', 'weekday_fl',
       'con_len_trans', 'text_len_trans', 'con_num_words_trans', 
        'text_num_words_trans',
       'time_since_first_trans', 'header_len_trans']
# Get pearson co-efficients
df[cols].corr()

for c in cols:
    print(c)
    print(df[np.isnan(df[col])].head(3))

score_cat
<0       6477
small    7324
big      6110
dtype: int64
score_trans
Empty DataFrame
Columns: [tId, cId, mob, vup, vdo, tit, aut, time, con, time_since_epoch, hour, weekday, weekday_fl, weekday_hour, is_answer, con_len, con_num_words, score, contr, article_id, updated, num_comments, link, header, sub, text, cat, header_len, text_len, text_num_words, time_since_first, score_cat, weekday_fl_trans, con_len_trans, text_len_trans, time_since_first_trans, score_trans, contr_trans, header_len_trans, con_num_words_trans, text_num_words_trans]
Index: []

[0 rows x 41 columns]
contr
Empty DataFrame
Columns: [tId, cId, mob, vup, vdo, tit, aut, time, con, time_since_epoch, hour, weekday, weekday_fl, weekday_hour, is_answer, con_len, con_num_words, score, contr, article_id, updated, num_comments, link, header, sub, text, cat, header_len, text_len, text_num_words, time_since_first, score_cat, weekday_fl_trans, con_len_trans, text_len_trans, time_since_first_trans, score_trans, contr_trans, h

### Prepare and split for model

In [106]:
# https://de.wikipedia.org/wiki/Liste_der_h%C3%A4ufigsten_W%C3%B6rter_der_deutschen_Sprache
stop_words = "die, der, und, in, zu, den, das, nicht, von, sie, ist, des, sich, mit, dem, dass, er, es, ein, ich, auf, so, eine, auch, als, an, nach, wie, im, für"
#stop_words += "man, aber, aus, durch, wenn, nur, war, noch, werden, bei, hat, wir, was, wird, sein, einen, welche, sind, oder, zur, um, haben, einer, mir, über, ihm, diese, einem, ihr, uns"
#stop_words += "da, zum, kann, doch, vor, dieser, mich, ihn, du, hatte, seine, mehr, am, denn, nun, unter, sehr, selbst, schon, hier"
#stop_words += "bis, habe, ihre, dann, ihnen, seiner, alle, wieder, meine, Zeit, gegen, vom, ganz, einzelnen, wo, muss, ohne, eines, können, sei"
stop_words = stop_words.lower()
stop_words = stop_words.split(', ')

def classify(val):
    if val > 10: return 1
    return 0

X = df.drop(['score', 'contr'], axis=1)

#y = df_merge_art['score'].apply(classify)
y = df['score']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Model 

In [107]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.linear_model import LinearRegression, Ridge
#from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression
#from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    '''
    Select a subset of data at a provided key.
    key: hashable, required
        The key corresponding to the desired value in a mappable.
    '''
    def __init__(self, key):
        self.key = key
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, df):
        return df[self.key]
    
# TODO onehotencoder for "cat"
no_numbers = lambda x: re.sub(r'(\d[\d\.])+', '', x.lower())
model = Pipeline([
    ('union', FeatureUnion(
        transformer_list=
        [
            ('statistics', Pipeline([
                ('selector', ItemSelector(
                    key=['weekday', 'hour', 'con_len_trans', 
                         'time_since_first_trans'])),
                ('scaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures(degree=5))
            ]))#,
            
            # Course of dimensionality!
         #   ('words', Pipeline([
          #      ('selector', ItemSelector(key='con')),
            #    ('vect', CountVectorizer(preprocessor=no_numbers, ngram_range=(1, 1), stop_words=stop_words))
           #     ('sent_model', sent_model)
           # ]))
        ],
        
        transformer_weights={
            'statistics': .8,
            'words': .2
        }
    )),
    
    # Params not optimized yet
    ('model', Ridge())
    #('model', SVC(kernel='linear', n_jobs=-1))
])

#model = make_pipeline(ItemSelector(
#        key=['weekday', 'hour', 'time_since_first', 'con_len']),
#        StandardScaler(), PolynomialFeatures(degree=5), Ridge(alpha=100, max_iter=10, solver='cholesky'))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("R^2: %1.3f\n" % r2_score(y_test, y_pred))
#print(classification_report(y_test, y_pred))

if False:
    from sklearn.model_selection import GridSearchCV
    param_grid = {'polynomialfeatures__degree': [2, 3, 4, 5],
        'ridge__alpha': np.logspace(-1, 4, 6), 'ridge__max_iter': [10, 50, 100], 'ridge__solver': ['svd', 'cholesky', 'lsqr']}
    grid = GridSearchCV(model, param_grid, cv=3)
    grid.fit(X_train, y_train)
    
    print("Best estimator:\n{}".format(grid.best_estimator_))

R^2: 0.056

